## Elektrifiseringen av sokkelen - Prosjektoppgave

#### Arbeidskrav 1

 1. Gjør en vurdering av tiltaker ihht. DFØs veilder og identifiser hvilken informasjon som trengs og hvilke forutsetninger som må gjøres. 

 2. Innhent informasjon 

 3. Sette opp en disposisjon/arbeidsplan (bruk veilederen)